In [15]:
import os
import numpy as np
import wfdb
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

segment_length = 1280
test_folder = "testing/"

def get_segment_label(ann, start_idx, end_idx):

    for i, sample in enumerate(ann.sample):
        if start_idx <= sample < end_idx:
            symbol = ann.symbol[i]
            if symbol != 'N':  
                return 1
    return 0

def load_test_data_and_labels(folder_path, segment_length=1280):
    segments = []
    labels = []
    for file in os.listdir(folder_path):
        if file.endswith(".hea"):
            record_path = os.path.join(folder_path, os.path.splitext(file)[0])
            try:
                record = wfdb.rdrecord(record_path)
                ann = wfdb.rdann(record_path, 'atr') 
                signal = record.p_signal[:, 0]

                for start in range(0, len(signal) - segment_length, segment_length):
                    end = start + segment_length
                    segment = signal[start:end]
                    segments.append(segment)
                    label = get_segment_label(ann, start, end)
                    labels.append(label)
            except Exception as e:
                print(f"Could not load {record_path}: {e}")
    if not segments:
        raise ValueError(f"No test data found in folder: {folder_path}")
    return np.array(segments), np.array(labels)

def preprocess_signal(segments):
    normalized = (segments - np.mean(segments, axis=1, keepdims=True)) / np.std(segments, axis=1, keepdims=True)
    return normalized[..., np.newaxis]


X_test_segments, y_true = load_test_data_and_labels(test_folder, segment_length)
X_test = preprocess_signal(X_test_segments)

print(f"Liczba segmentów: {len(y_true)}")
print(f"Zdrowe segmenty: {(y_true == 0).sum()}")
print(f"Chore segmenty: {(y_true == 1).sum()}")

for i, (label, prob) in enumerate(zip(y_true[:10], y_pred_probs[:10].flatten())):
    print(f"Segment {i}: Label={label}, Predicted_prob={prob:.3f}")


model = load_model("models/ekg-anomaly-classifier.keras")

y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.3).astype(int).flatten()

acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)
cm = confusion_matrix(y_true, y_pred, labels=[0,1])

print("Results")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:")
print(cm)


Liczba segmentów: 13962
Zdrowe segmenty: 6607
Chore segmenty: 7355
Segment 0: Label=0, Predicted_prob=1.000
Segment 1: Label=0, Predicted_prob=1.000
Segment 2: Label=1, Predicted_prob=1.000
Segment 3: Label=0, Predicted_prob=1.000
Segment 4: Label=0, Predicted_prob=1.000
Segment 5: Label=0, Predicted_prob=1.000
Segment 6: Label=0, Predicted_prob=1.000
Segment 7: Label=0, Predicted_prob=1.000
Segment 8: Label=0, Predicted_prob=1.000
Segment 9: Label=0, Predicted_prob=1.000
437/437 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step
Results
Accuracy: 0.5753
Precision: 0.6042
Recall: 0.5617
F1-score: 0.5822
Confusion Matrix:
[[3901 2706]
 [3224 4131]]
